Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---
<p style="padding: 10px; border: 2px solid red;">
    <b>Before you start:</b> Today is the day you should submit the dataset for your Unit 2 Build Week project. You can review the guidelines and make your submission in the Build Week course for your cohort on Canvas.</p>

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/main/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [3]:
def color_percent(floaty):
  "Input a normalized number between 0 and 1, and get it in string form colored according to percentage."
  if floaty >= .5:
    green = 200
    red = int(500 * (1 - floaty))
  else:
    red = 255
    green = int(400 * floaty)

  return f'\033[38;2;{red};{green};0m{floaty}\033[00m'

# Module Project: Hyperparameter Tuning

This sprint, the module projects will focus on creating and improving a model for the Tanazania Water Pump dataset. Your goal is to create a model to predict whether a water pump is functional, non-functional, or needs repair.

Dataset source: [DrivenData.org](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/).

## Directions

The tasks for this project are as follows:

- **Task 1:** Use `wrangle` function to import training and test data.
- **Task 2:** Split training data into feature matrix `X` and target vector `y`.
- **Task 3:** Establish the baseline accuracy score for your dataset.
- **Task 4:** Build `clf_dt`.
- **Task 5:** Build `clf_rf`.
- **Task 6:** Evaluate classifiers using k-fold cross-validation.
- **Task 7:** Tune hyperparameters for best performing classifier.
- **Task 8:** Print out best score and params for model.
- **Task 9:** Create `submission.csv` and upload to Kaggle.

You should limit yourself to the following libraries for this project:

- `category_encoders`
- `matplotlib`
- `pandas`
- `pandas-profiling`
- `sklearn`

# I. Wrangle Data

In [7]:
import pandas as pd

def wrangle(fm_path, tv_path=None):
    if tv_path:
        df = pd.merge(pd.read_csv(fm_path, 
                                  na_values=[0, -2.000000e-08]),
                      pd.read_csv(tv_path)).set_index('id')
    else:
        df = pd.read_csv(fm_path, 
                         na_values=[0, -2.000000e-08],
                         index_col='id')

    # Drop constant columns
    df.drop(columns=['recorded_by'], inplace=True)

    # Drop HCCCs
    cutoff = 100
    drop_cols = [col for col in df.select_dtypes('object').columns
                 if df[col].nunique() > cutoff]
    df.drop(columns=drop_cols, inplace=True)

    # Drop duplicate columns
    dupe_cols = [col for col in df.head(15).T.duplicated().index
                 if df.head(15).T.duplicated()[col]]
    df.drop(columns=dupe_cols, inplace=True)             

    return df

**Task 1:** Using the above `wrangle` function to read `train_features.csv` and `train_labels.csv` into the DataFrame `df`, and `test_features.csv` into the DataFrame `X_test`.

In [9]:
df = wrangle('train_features.csv', 'train_labels.csv')
X_test = wrangle('test_features.csv')

# II. Split Data

**Task 2:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'status_group'`.

**Note:** You won't need to do a train-test split because you'll use cross-validation instead.

In [11]:
target = 'status_group'
X = df.drop(columns=target)
y = df[target]

# III. Establish Baseline

**Task 3:** Since this is a **classification** problem, you should establish a baseline accuracy score. Figure out what is the majority class in `y_train` and what percentage of your training observations it represents.

In [12]:
baseline_acc = y.value_counts(normalize=True).max()
print('Baseline Accuracy Score:', color_percent(baseline_acc))

Baseline Accuracy Score: 0.5429828068772491


# IV. Build Models

**Task 4:** Build a `Pipeline` named `clf_dt`. Your `Pipeline` should include:

- an `OrdinalEncoder` transformer for categorical features.
- a `SimpleImputer` transformer fot missing values.
- a `DecisionTreeClassifier` Predictor.

**Note:** Do not train `clf_dt`. You'll do that in a subsequent task. 

In [13]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

In [14]:
clf_dt = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    DecisionTreeClassifier(
        random_state=42
    )
)

**Task 5:** Build a `Pipeline` named `clf_rf`. Your `Pipeline` should include:

- an `OrdinalEncoder` transformer for categorical features.
- a `SimpleImputer` transformer fot missing values.
- a `RandomForestClassifier` predictor.

**Note:** Do not train `clf_rf`. You'll do that in a subsequent task. 

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
clf_rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(
        random_state=42,
        n_jobs=-1,
        criterion='entropy',
        max_depth=25,
        n_estimators=164,
        min_samples_leaf=2
    )
)

# V. Check Metrics

**Task 6:** Evaluate the performance of both of your classifiers using k-fold cross-validation.

In [18]:
from sklearn.model_selection import cross_val_score

In [39]:
cv_scores_dt = cross_val_score(clf_dt, X, y, cv=5)
cv_scores_rf = cross_val_score(clf_rf, X, y, cv=5, n_jobs=-1)

In [40]:
print('CV scores DecisionTreeClassifier')
print(cv_scores_dt)
print('Mean CV accuracy score:', color_percent(cv_scores_dt.mean()))
print('STD CV accuracy score:', cv_scores_dt.std())

CV scores DecisionTreeClassifier
[0.74137205 0.75094697 0.74789562 0.75147306 0.74565927]
Mean CV accuracy score: 0.7474693951865635
STD CV accuracy score: 0.003708333827823888


In [41]:
print('CV score RandomForestClassifier')
print(cv_scores_rf)
print('Mean CV accuracy score:', color_percent(cv_scores_rf.mean()))
print('STD CV accuracy score:', cv_scores_rf.std())

CV score RandomForestClassifier
[0.80787037 0.80713384 0.8052399  0.81302609 0.80479848]
Mean CV accuracy score: 0.8076137373418495
STD CV accuracy score: 0.002937704107809573


# VI. Tune Model

**Task 7:** Choose the best performing of your two models and tune its hyperparameters using a `RandomizedSearchCV` named `model`. Make sure that you include cross-validation and that `n_iter` is set to at least `25`.

**Note:** If you're not sure which hyperparameters to tune, check the notes from today's guided project and the `sklearn` documentation. 

In [27]:
from sklearn.model_selection import GridSearchCV

In [25]:
model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

In [35]:
param_grid = {
    'simpleimputer__strategy': ['median'],
    'randomforestclassifier__max_depth': [25],
    'randomforestclassifier__n_estimators': range(155, 166, 1),
    'randomforestclassifier__min_samples_leaf': [2],
    'randomforestclassifier__criterion': ['entropy']
}

In [36]:
model_rfgs = GridSearchCV(
    model,
    param_grid=param_grid,
    n_jobs=-1,
    cv=5,
    verbose=1
)

model_rfgs.fit(X,y)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed: 11.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=None,
                                                       drop_invariant=False,
                                                       handle_missing='value',
                                                       handle_unknown='value',
                                                       mapping=None,
                                                       return_df=True,
                                                       verbose=0)),
                                       ('simpleimputer',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
          

**Task 8:** Print out the best score and best params for `model`.

In [37]:
best_score = model_rfgs.best_score_
best_params = model_rfgs.best_params_

print('Best score for `model`:', color_percent(best_score))
print('Best params for `model`:', best_params)

# Best score for `model`: 0.8073191223328022
# Best params for `model`: {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 25, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__n_estimators': 150, 'simpleimputer__strategy': 'median'}

# Best score for `model`: 0.8074874636434144
# Best params for `model`: {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 25, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__n_estimators': 160, 'simpleimputer__strategy': 'median'}

# Best score for `model`: 0.8076137373418495
# Best params for `model`: {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 25, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__n_estimators': 164, 'simpleimputer__strategy': 'median'}

Best score for `model`: 0.8076137373418495
Best params for `model`: {'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 25, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__n_estimators': 164, 'simpleimputer__strategy': 'median'}


# Communicate Results

**Task 9:** Create a DataFrame `submission` whose index is the same as `X_test` and that has one column `'status_group'` with your predictions. Next, save this DataFrame as a CSV file and upload your submissions to our competition site. 

**Note:** Check the `sample_submission.csv` file on the competition website to make sure your submissions follows the same formatting. 

In [46]:
X_test.drop(columns=['waterpoint_type_group'], inplace=True)

In [ ]:
clf_rf.fit(X, y)


In [47]:
y_pred = clf_rf.predict(X_test)
submission = pd.DataFrame({'status_group':y_pred}, index=X_test.index)
datestamp = pd.Timestamp.now().strftime('%Y-%m-%d_%H%M_')
submission.to_csv(f'{datestamp}submission.csv')